## Process CLUM dataset

Extracting plantation forests for masking

Attribute table: https://di-daa.img.arcgis.com/arcgis/rest/services/Land_and_vegetation/Catchment_Scale_Land_Use_Secondary/ImageServer/rasterAttributeTable


In [ ]:
import numpy as np
import xarray as xr
import rioxarray as rxr
from odc.geo.xr import assign_crs
import matplotlib.pyplot as plt

import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import start_local_dask, round_coords

In [ ]:
client = start_local_dask()
client

In [ ]:
# Path to raster
path = "/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/clum_50m_2023.tiff"

# Load from file
ds = rxr.open_rasterio(
    path,
    masked=True,
    chunks={"x": 200, "y": 200},
)

#reproject
ds_path = '/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/AusEFlux_versions/AusEFlux_GPP_5km_1982_2022_v0.5.nc'
gbox = assign_crs(xr.open_dataset(ds_path), crs='EPSG:4326').odc.geobox

ds = ds.odc.reproject(how=gbox, resampling='mode')
ds = ds.rename('CLUM_5km')
ds = round_coords(ds)
ds = ds.squeeze().drop_vars('band')
ds = ds.where(ds>0)
ds.to_netcdf('/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/clum_50m_2023_5km.nc')

plantation_forests = xr.where(ds.isin([310,311,312,313,314]), 1, 0)
plantation_forests.to_netcdf('/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/plantation_forests_5km.nc')

In [ ]:
# ds.where(ds.isin([310,311,312,313, 314])).odc.explore(
#         tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#     attr = 'Esri',
#     name = 'Esri Satellite',
#     cmap='tab20',
#     robust=False
# )